<a href="https://colab.research.google.com/github/anshulk-cmu/CUDA_PageRank/blob/main/CUDA_PageRank_(Google_Search_Engine).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# GPU env check & log (Colab)
import os, sys, subprocess, datetime, platform
log = []

from datetime import datetime, UTC
ts = datetime.now(UTC).isoformat()

def run(cmd):
    try:
        out = subprocess.check_output(cmd, shell=True, stderr=subprocess.STDOUT, text=True)
    except subprocess.CalledProcessError as e:
        out = e.output
    return out.strip()

log.append(f"Timestamp: {datetime.now(UTC).isoformat()}Z")
log.append(f"Python: {sys.version.split()[0]}  OS: {platform.platform()}")

# nvidia-smi
log.append("=== nvidia-smi ===")
log.append(run("nvidia-smi || echo 'nvidia-smi not found'"))

# nvcc
log.append("=== nvcc --version ===")
log.append(run("nvcc --version || echo 'nvcc not found'"))

# PyTorch CUDA
try:
    import torch
    log.append(f"PyTorch: {torch.__version__}")
    log.append(f"torch.cuda.is_available: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        i = torch.cuda.current_device()
        props = torch.cuda.get_device_properties(i)
        gb = props.total_memory / (1024**3)
        log.append(f"CUDA device count: {torch.cuda.device_count()}")
        log.append(f"Current device index: {i}")
        log.append(f"Device name: {torch.cuda.get_device_name(i)}")
        log.append(f"Compute capability: {props.major}.{props.minor}")
        log.append(f"Total memory (GB): {gb:.2f}")
        log.append(f"CUDA runtime version (torch): {torch.version.cuda}")
    else:
        log.append("CUDA not available in this runtime. Enable GPU in Runtime → Change runtime type.")
except Exception as e:
    log.append(f"PyTorch check failed: {e}")

# Print and save
text = "\n".join(log)
print(text)
with open("/content/gpu_env_log.txt", "w") as f:
    f.write(text)

Timestamp: 2025-09-28T01:03:28.635306+00:00Z
Python: 3.12.11  OS: Linux-6.6.97+-x86_64-with-glibc2.35
=== nvidia-smi ===
Sun Sep 28 01:03:28 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   56C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                  

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

!mkdir -p /content/drive/MyDrive/cuda-pagerank/data
%cd /content/drive/MyDrive/cuda-pagerank/data

# Download SNAP web-Google (directed web graph)
!wget -nc https://snap.stanford.edu/data/web-Google.txt.gz
!gunzip -kf web-Google.txt.gz

# Quick sanity: show first lines and basic counts
!head -n 5 web-Google.txt
!grep -v '^\s*#' web-Google.txt | wc -l

Mounted at /content/drive
/content/drive/MyDrive/cuda-pagerank/data
--2025-09-28 01:04:07--  https://snap.stanford.edu/data/web-Google.txt.gz
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21168784 (20M) [application/x-gzip]
Saving to: ‘web-Google.txt.gz’

web-Google.txt.gz   100%[===================>]  20.19M  5.21MB/s    in 7.5s    

2025-09-28 01:04:16 (2.69 MB/s) - ‘web-Google.txt.gz’ saved [21168784/21168784]

# Directed graph (each unordered pair of nodes is saved once): web-Google.txt 
# Webgraph from the Google programming contest, 2002
# Nodes: 875713 Edges: 5105039
# FromNodeId	ToNodeId
0	11342
5105039


In [4]:
# Build in-neighbor CSR (rows = dest nodes, cols = source nodes), plus out-degree
import numpy as np, os, gzip, io, time, json

data_path = "/content/drive/MyDrive/cuda-pagerank/data/web-Google.txt"
assert os.path.exists(data_path)

# 1) read edges (skip '#')
src = []
dst = []
with open(data_path, "r") as f:
    for line in f:
        if line.startswith("#"):
            continue
        a,b = line.strip().split()
        src.append(int(a)); dst.append(int(b))
src = np.array(src, dtype=np.int64)
dst = np.array(dst, dtype=np.int64)

N = int(max(src.max(), dst.max())) + 1
M = len(src)

# 2) out-degree (for weights 1/outdeg)
outdeg = np.bincount(src, minlength=N).astype(np.int64)

# 3) in-CSR row_ptr/col_idx/val (val = 1/outdeg(source); dangling nodes => 0 later)
row_ptr = np.zeros(N + 1, dtype=np.int64)
np.add.at(row_ptr, dst + 1, 1)
np.cumsum(row_ptr, out=row_ptr)

col_idx = np.empty(M, dtype=np.int64)
val = np.empty(M, dtype=np.float64)

# fill per row using a cursor
cursor = row_ptr[:-1].copy()
w = 1.0 / np.maximum(outdeg, 1)  # avoid div-by-zero
for s, d in zip(src, dst):
    i = cursor[d]
    col_idx[i] = s
    val[i] = w[s]
    cursor[d] = i + 1

# optional: sort columns within each row for better coalescing later
for r in range(N):
    start, end = row_ptr[r], row_ptr[r+1]
    if end - start > 1:
        idx = np.argsort(col_idx[start:end], kind="mergesort")
        col_idx[start:end] = col_idx[start:end][idx]
        val[start:end] = val[start:end][idx]

# persist
outdir = "/content/drive/MyDrive/cuda-pagerank/prep"
os.makedirs(outdir, exist_ok=True)
np.savez_compressed(f"{outdir}/in_csr_webGoogle.npz",
                    N=N, M=M, row_ptr=row_ptr, col_idx=col_idx, val=val, outdeg=outdeg)

# quick log
print(json.dumps({
    "nodes": int(N),
    "edges": int(M),
    "dangling_count": int((outdeg==0).sum()),
    "sample_row0_len": int(row_ptr[1]-row_ptr[0]),
    "saved": f"{outdir}/in_csr_webGoogle.npz"
}, indent=2))

{
  "nodes": 916428,
  "edges": 5105039,
  "dangling_count": 176974,
  "sample_row0_len": 212,
  "saved": "/content/drive/MyDrive/cuda-pagerank/prep/in_csr_webGoogle.npz"
}


In [5]:
# CPU PageRank (power method, minimal)
import numpy as np, time, json

npz = np.load("/content/drive/MyDrive/cuda-pagerank/prep/in_csr_webGoogle.npz", allow_pickle=False)
N = int(npz["N"]); M = int(npz["M"])
row_ptr = npz["row_ptr"]; col_idx = npz["col_idx"]; val = npz["val"]; outdeg = npz["outdeg"]

alpha = 0.85
tol = 1e-6
max_iter = 200

r = np.full(N, 1.0 / N, dtype=np.float64)
r_new = np.empty_like(r)

dangling_mask = (outdeg == 0)

def spmv_pull(row_ptr, col_idx, val, x):
    y = np.zeros_like(x)
    for i in range(N):
        s = 0.0
        start, end = row_ptr[i], row_ptr[i+1]
        for p in range(start, end):
            s += val[p] * x[col_idx[p]]
        y[i] = s
    return y

t0 = time.time()
for it in range(1, max_iter + 1):
    tmp = spmv_pull(row_ptr, col_idx, val, r)
    dangling_mass = r[dangling_mask].sum() / N
    r_new[:] = alpha * (tmp + dangling_mass) + (1.0 - alpha) / N
    diff = np.abs(r_new - r).sum()
    r, r_new = r_new, r
    if diff < tol:
        break
t1 = time.time()

print(json.dumps({
    "N": N, "M": M,
    "iterations": it,
    "residual_L1": float(diff),
    "rank_sum": float(r.sum()),
    "time_sec": round(t1 - t0, 3)
}, indent=2))

{
  "N": 916428,
  "M": 5105039,
  "iterations": 62,
  "residual_L1": 8.729468467677312e-07,
  "rank_sum": 0.9999999999999994,
  "time_sec": 218.386
}


In [6]:
# Save CSR to flat binaries for CUDA C++ (float64 weights)
import numpy as np, json, os

npz = np.load("/content/drive/MyDrive/cuda-pagerank/prep/in_csr_webGoogle.npz", allow_pickle=False)
N = int(npz["N"]); M = int(npz["M"])
row_ptr = npz["row_ptr"].astype(np.int64, copy=False)
col_idx = npz["col_idx"].astype(np.int64, copy=False)
val = npz["val"].astype(np.float64, copy=False)
outdeg = npz["outdeg"].astype(np.int64, copy=False)

outdir = "/content/drive/MyDrive/cuda-pagerank/bin"
os.makedirs(outdir, exist_ok=True)

row_ptr.tofile(f"{outdir}/row_ptr.bin")
col_idx.tofile(f"{outdir}/col_idx.bin")
val.tofile(f"{outdir}/val.bin")
outdeg.tofile(f"{outdir}/outdeg.bin")

with open(f"{outdir}/meta.json", "w") as f:
    json.dump({"N": N, "M": M, "alpha": 0.85, "tol": 1e-6, "max_iter": 200}, f)

print("Saved:", outdir)

Saved: /content/drive/MyDrive/cuda-pagerank/bin


In [13]:
# Write CUDA source with timing + GB/s
%%writefile /content/pagerank_pull.cu
#include <cstdio>
#include <cstdlib>
#include <cstdint>
#include <vector>
#include <fstream>
#include <thrust/device_vector.h>
#include <thrust/reduce.h>
#include <thrust/transform_reduce.h>
#include <thrust/transform.h>
#include <thrust/functional.h>
#include <thrust/iterator/zip_iterator.h>
#include <thrust/iterator/counting_iterator.h>
#include <thrust/tuple.h>
#include <cmath>
#include <cuda_runtime.h>

struct AbsDiff {
  __host__ __device__ double operator()(const thrust::tuple<double,double>& a) const {
    return fabs(thrust::get<0>(a) - thrust::get<1>(a));
  }
};
struct MaskPick {
  const int64_t* outdeg;
  __host__ __device__ double operator()(const thrust::tuple<double,int64_t>& t) const {
    return (thrust::get<1>(t)==0) ? thrust::get<0>(t) : 0.0;
  }
};

__global__ void spmv_pull_kernel(const int64_t* __restrict__ row_ptr,
                                 const int64_t* __restrict__ col_idx,
                                 const double*  __restrict__ val,
                                 const double*  __restrict__ r,
                                 double*        __restrict__ tmp,
                                 int64_t N){
  int64_t i = blockIdx.x * blockDim.x + threadIdx.x;
  if (i >= N) return;
  int64_t s0 = row_ptr[i], s1 = row_ptr[i+1];
  double s = 0.0;
  for (int64_t p = s0; p < s1; ++p) s += val[p] * r[col_idx[p]];
  tmp[i] = s;
}

__global__ void finalize_kernel(double* __restrict__ r_new,
                                const double* __restrict__ tmp,
                                double alpha, double d_ave, double base,
                                int64_t N){
  int64_t i = blockIdx.x * blockDim.x + threadIdx.x;
  if (i >= N) return;
  r_new[i] = alpha * (tmp[i] + d_ave) + base;
}

static void read_bin(const char* path, void* buf, size_t bytes){
  std::ifstream f(path, std::ios::binary);
  if(!f){ fprintf(stderr,"Cannot open %s\n", path); std::exit(1); }
  f.read(reinterpret_cast<char*>(buf), bytes);
  if(!f){ fprintf(stderr,"Short read on %s\n", path); std::exit(1); }
}

int main(){
  printf("=== CUDA PageRank Implementation ===\n");

  // tiny JSON parse
  printf("Loading configuration...\n");
  std::ifstream mf("/content/drive/MyDrive/cuda-pagerank/bin/meta.json");
  std::string s((std::istreambuf_iterator<char>(mf)), std::istreambuf_iterator<char>());
  auto getnum=[&](const char* key)->double{
    auto k=s.find(key); if(k==std::string::npos) return 0; k=s.find(':',k); auto e=s.find_first_of(",}\n",k+1);
    return atof(s.substr(k+1,e-k-1).c_str());
  };
  int64_t N=(int64_t)getnum("\"N\""), M=(int64_t)getnum("\"M\"");
  double alpha=getnum("\"alpha\""), tol=getnum("\"tol\"");
  int max_iter=(int)getnum("\"max_iter\"");

  printf("Problem Size:\n");
  printf("  Nodes (web pages): %lld\n", (long long)N);
  printf("  Edges (links):     %lld\n", (long long)M);
  printf("  Avg links/page:    %.2f\n", (double)M / (double)N);
  printf("Parameters:\n");
  printf("  Damping factor:    %.3f\n", alpha);
  printf("  Tolerance:         %.1e\n", tol);
  printf("  Max iterations:    %d\n", max_iter);
  printf("\n");

  // host
  printf("Loading graph data from disk...\n");
  cudaEvent_t load_start, load_end;
  cudaEventCreate(&load_start); cudaEventCreate(&load_end);
  cudaEventRecord(load_start);

  std::vector<int64_t> h_row_ptr(N+1), h_col_idx(M), h_outdeg(N);
  std::vector<double>  h_val(M);
  read_bin("/content/drive/MyDrive/cuda-pagerank/bin/row_ptr.bin", h_row_ptr.data(), (N+1)*sizeof(int64_t));
  read_bin("/content/drive/MyDrive/cuda-pagerank/bin/col_idx.bin", h_col_idx.data(), M*sizeof(int64_t));
  read_bin("/content/drive/MyDrive/cuda-pagerank/bin/val.bin",     h_val.data(),     M*sizeof(double));
  read_bin("/content/drive/MyDrive/cuda-pagerank/bin/outdeg.bin",  h_outdeg.data(),  N*sizeof(int64_t));

  // device
  printf("Transferring data to GPU...\n");
  thrust::device_vector<int64_t> d_row_ptr = h_row_ptr;
  thrust::device_vector<int64_t> d_col_idx = h_col_idx;
  thrust::device_vector<int64_t> d_outdeg  = h_outdeg;
  thrust::device_vector<double>  d_val     = h_val;
  thrust::device_vector<double>  r(N, 1.0/(double)N), r_new(N, 0.0), tmp(N, 0.0);

  cudaEventRecord(load_end);
  cudaEventSynchronize(load_end);
  float load_ms;
  cudaEventElapsedTime(&load_ms, load_start, load_end);

  // Calculate memory usage
  double total_mb = ((N+1 + M + N) * sizeof(int64_t) + M * sizeof(double) + 3*N * sizeof(double)) / (1024.0 * 1024.0);
  printf("Memory transfer completed in %.1f ms (%.1f MB total)\n", load_ms, total_mb);

  const int64_t threads = 256;
  const int64_t blocks  = (N + threads - 1) / threads;
  const double base = (1.0 - alpha) / (double)N;

  printf("GPU Configuration:\n");
  printf("  Threads per block: %lld\n", (long long)threads);
  printf("  Total blocks:      %lld\n", (long long)blocks);
  printf("\n");

  auto zip_dang  = thrust::make_zip_iterator(thrust::make_tuple(r.begin(), d_outdeg.begin()));
  auto zip_rpair = thrust::make_zip_iterator(thrust::make_tuple(r.begin(), r_new.begin()));

  // timing
  cudaEvent_t e0,e1; cudaEventCreate(&e0); cudaEventCreate(&e1);
  double residual=0.0; int it=0; double ms_total=0.0;

  printf("Starting PageRank iterations...\n");
  printf("Iter | Residual (L1) | Time (ms) | Status\n");
  printf("-----|---------------|-----------|--------\n");

  for(it=1; it<=max_iter; ++it){
    cudaEventRecord(e0);
    spmv_pull_kernel<<<(unsigned)blocks,(unsigned)threads>>>(thrust::raw_pointer_cast(d_row_ptr.data()),
                                                             thrust::raw_pointer_cast(d_col_idx.data()),
                                                             thrust::raw_pointer_cast(d_val.data()),
                                                             thrust::raw_pointer_cast(r.data()),
                                                             thrust::raw_pointer_cast(tmp.data()),
                                                             N);
    cudaDeviceSynchronize();
    MaskPick pick{ thrust::raw_pointer_cast(d_outdeg.data()) };
    double d_mass = thrust::transform_reduce(zip_dang, zip_dang + N, pick, 0.0, thrust::plus<double>());
    double d_ave  = d_mass / (double)N;
    finalize_kernel<<<(unsigned)blocks,(unsigned)threads>>>(thrust::raw_pointer_cast(r_new.data()),
                                                            thrust::raw_pointer_cast(tmp.data()),
                                                            alpha, d_ave, base, N);
    cudaDeviceSynchronize();
    residual = thrust::transform_reduce(zip_rpair, zip_rpair + N, AbsDiff(), 0.0, thrust::plus<double>());
    r.swap(r_new);
    cudaEventRecord(e1); cudaEventSynchronize(e1);
    float ms=0.0f; cudaEventElapsedTime(&ms,e0,e1);
    ms_total += ms;

    // Progress logging
    if (it == 1 || it % 10 == 0 || residual < tol) {
      printf("%4d | %12.6e | %8.3f | %s\n",
             it, residual, ms,
             (residual < tol) ? "CONVERGED" : "Running");
    }

    if (residual < tol) break;
  }

  double sum_r = thrust::reduce(r.begin(), r.end(), 0.0, thrust::plus<double>());

  printf("\n=== Results Summary ===\n");
  printf("Convergence:       %s after %d iterations\n",
         (residual < tol) ? "SUCCESS" : "MAX_ITER_REACHED", it);
  printf("Final residual:    %.6e\n", residual);
  printf("Rank sum:          %.12f (should be ~1.0)\n", sum_r);
  printf("Total time:        %.3f ms\n", ms_total);
  printf("Avg time/iter:     %.3f ms\n", ms_total / it);

  // rough bytes/iter: edges*(val+col_idx+r) + nodes*(tmp+r_new+r old ~ 3 doubles)
  const double bytes_per_edge = 8.0 + 8.0 + 8.0;   // val + col_idx + r[j]
  const double bytes_per_node = 8.0 + 8.0 + 8.0;   // tmp[i] + r_new[i] + r[i]
  double ms_per_iter = ms_total / it;
  double bytes = (bytes_per_edge * (double)M + bytes_per_node * (double)N);
  double gbs   = (bytes / (ms_per_iter/1e3)) / 1e9;

  printf("Memory bandwidth:  %.2f GB/s\n", gbs);
  printf("\n");

  // JSON output for programmatic use
  printf("JSON_RESULT: {\"N\":%lld,\"M\":%lld,\"iterations\":%d,"
         "\"residual_L1\":%.12e,\"rank_sum\":%.15f,"
         "\"ms_per_iter\":%.3f,\"est_GBps\":%.2f,\"total_ms\":%.3f}\n",
         (long long)N,(long long)M,it,residual,sum_r,ms_per_iter,gbs,ms_total);

  cudaEventDestroy(e0);
  cudaEventDestroy(e1);
  cudaEventDestroy(load_start);
  cudaEventDestroy(load_end);

  return 0;
}

Overwriting /content/pagerank_pull.cu


In [14]:
# Compile for T4 or generic (sm_75 fits T4).
!nvcc -O3 -arch=sm_75 /content/pagerank_pull.cu -o /content/pagerank_pull

In [15]:
# Run the CUDA binary (reads from /drive/.../bin), prints JSON summary
!/content/pagerank_pull

=== CUDA PageRank Implementation ===
Loading configuration...
Problem Size:
  Nodes (web pages): 916428
  Edges (links):     5105039
  Avg links/page:    5.57
Parameters:
  Damping factor:    0.850
  Tolerance:         1.0e-06
  Max iterations:    200

Loading graph data from disk...
Transferring data to GPU...
Memory transfer completed in 197.7 ms (112.9 MB total)
GPU Configuration:
  Threads per block: 256
  Total blocks:      3580

Starting PageRank iterations...
Iter | Residual (L1) | Time (ms) | Status
-----|---------------|-----------|--------
   1 | 8.508467e-01 |    3.668 | Running
  10 | 1.329880e-02 |    3.437 | Running
  20 | 1.827460e-03 |    3.445 | Running
  30 | 3.289641e-04 |    3.402 | Running
  40 | 6.311191e-05 |    3.221 | Running
  50 | 1.240221e-05 |    3.244 | Running
  60 | 2.463703e-06 |    3.294 | Running
  66 | 9.360145e-07 |    3.204 | CONVERGED

=== Results Summary ===
Convergence:       SUCCESS after 66 iterations
Final residual:    9.360145e-07
Rank sum: 